In [ ]:
from pyspark.sql.types import IntegerType

In [ ]:
purchasePath = "data/Purchases.txt"
# usersPath = "data/Users.txt"
cataloguePath = "data/Catalogue.txt"

outputPath1 = "outPart1SQL/"
outputPath2 = "outPart2SQL/"

In [ ]:
# Define the dataframes and table associated with Purchases and Catalogue

# SaleTimestamp,UserID,ItemID,SalePrice
purchaseDF = spark.read.format("csv")\
                    .option("header", True)\
                    .option("inferSchema", True)\
                    .load(purchasePath)

# Register the temporary table purchases
purchaseDF.createOrReplaceTempView("purchases")


# ItemID,Name,Category,StillInProduction
catalogueDF = spark.read.format("csv")\
                    .option("header", True)\
                    .option("inferSchema", True)\
                    .load(cataloguePath)

# Register the temporary table catalogue
catalogueDF.createOrReplaceTempView("catalogue")

In [ ]:
######################
# PART 1
######################

In [ ]:
# Define a python function that given a timestamp and a year 
# returns 1 if the timestamp is associated with year. Otherwise, 0
def timestampMatchYear(timestamp, year):
    yearTimestamp = timestamp.split('/')[0]

    if (yearTimestamp==year):
        return 1
    else:
        return 0


# Register a UDF associated with timestampMatchYear
spark.udf.register("Match_Year", \
                   timestampMatchYear, \
                   IntegerType())

In [ ]:
# Select the purchases associated with 2022 or 2023
# This subset of records is helpful for both sub-tasksof this problem
purchases2223DF = spark.sql("""SELECT * """+
                            """FROM purchases """+
                            """WHERE SaleTimestamp LIKE '2022%' """ +
                            """   OR SaleTimestamp LIKE '2023%'""")\
                        .cache()

# Register the temporary table purchases2223
purchases2223DF.createOrReplaceTempView("purchases2223")

In [ ]:
# Compute the number of purchases for each customers in 2022 and 2023
#
# The UDF is used two times. 
# -- First, to create a first temporary column that is 1 if the year is 2022; othewise, 0.
# -- Then, to create a second temporary column that is 1 if the year is 2023; othewise, 0.
#
# Using GROUP BY and the SUM function applied to the tempraty columns, with one single scan of the table,
# we compute the number of purchases in 2022 and 2023 for each user

userCountPurchases2223DF = spark.sql("""SELECT UserID, """+
                                     """SUM(Match_Year(SaleTimestamp, '2022')) as NumPurchases2022, """+
                                     """SUM(Match_Year(SaleTimestamp, '2023')) as NumPurchases2023 """+
                                     """FROM purchases2223 """+
                                     """GROUP BY UserID """)\
                                .cache()

# Register the temporary table userCountPurchases2223
userCountPurchases2223DF.createOrReplaceTempView("userCountPurchases2223")

In [ ]:
# Compute for each year 2022 and 2023, the maximum number of purchases among all users
maxPurchases2223DF = spark.sql("""SELECT MAX(NumPurchases2022) as Max2022, """+
                               """       MAX(NumPurchases2023) as Max2023 """+
                               """FROM userCountPurchases2223 """)

# Register the temporary table maxPurchases2223
maxPurchases2223DF.createOrReplaceTempView("maxPurchases2223")

In [ ]:
# Join userCountPurchases2223 with maxPurchases2223 to keep the users associated with the 
# highest number of purchases in the years 2022 or 2023

res1DF = spark.sql("""SELECT UserID """ +
                   """FROM userCountPurchases2223, maxPurchases2223 """+
                   """WHERE userCountPurchases2223.NumPurchases2022 = Max2022 """+
                   """   OR userCountPurchases2223.NumPurchases2023 = Max2023 """)

In [ ]:
# Store the result of Part 1
res1DF.write\
    .format("csv")\
    .option("header", False)\
    .save(outputPath1)

In [ ]:
######################
# PART 2 - v1
######################

In [ ]:
# Considering the purchases in year 2022/2023 (purchases2223),
# count for each ItemID the number of (distinct) users who bought that item

itemDistinctUsersPurchasesDF = spark.sql("""select ItemID, count(DISTINCT UserID) as DistinctUsers """ +
                                         """FROM purchases2223 """+
                                         """GROUP BY ItemID""")

# Register the temporary table itemDistinctUsersPurchases
itemDistinctUsersPurchasesDF.createOrReplaceTempView("itemDistinctUsersPurchases")

In [ ]:
# for each item, retrieve the corresponding category from catalogue
itemCategoryPurchasesDF = spark.sql("""select itemDistinctUsersPurchases.*, """ +
                                    """ catalogue.Category """ +
                                    """FROM itemDistinctUsersPurchases, catalogue """+
                                    """WHERE itemDistinctUsersPurchases.ItemID=catalogue.ItemID""")\
                                .cache()

# Register the temporary table itemCategoryPurchases
itemCategoryPurchasesDF.createOrReplaceTempView("itemCategoryPurchases")

In [ ]:
# compute for each category the maximum number of distinct users who purchased the item


maxDistinctUsersPurchasesPerCategoryDF = spark.sql("""select Category, MAX(DistinctUsers) as MaxUsers """ +
                                                   """FROM itemCategoryPurchases """ +
                                                   """GROUP BY Category """)


# Register the temporary table maxDistinctUsersPurchasesPerCategory
maxDistinctUsersPurchasesPerCategoryDF.createOrReplaceTempView("maxDistinctUsersPurchasesPerCategory")

In [ ]:
# Join itemCategoryPurchases with maxDistinctUsersPurchasesPerCategory 
# to keep only the items associated with the maximum number of purchases for its category.
# 
# Finally, select the attributes category and itemID

res2PartialDF = spark.sql("""SELECT itemCategoryPurchases.Category, itemCategoryPurchases.ItemID """ +
                          """FROM itemCategoryPurchases, maxDistinctUsersPurchasesPerCategory """ +
                          """WHERE itemCategoryPurchases.Category=maxDistinctUsersPurchasesPerCategory.Category """+
                          """  AND itemCategoryPurchases.DistinctUsers=maxDistinctUsersPurchasesPerCategory.MaxUsers """)

# Register the temporary table res2Partial
res2PartialDF.createOrReplaceTempView("res2Partial")

In [ ]:
# from res2Partial we need to add the 0-case, i.e., categories with items which were never purchased.
# Consider all distinct categories and subtract those in res2Partial.
#
# At the end, "map" the selected categories to records category, 'NoPurchases'


unsoldCategoriesDF = spark.sql("""SELECT DISTINCT category, 'NoPurchases' """ +
                               """FROM catalogue """ +
                               """WHERE category NOT IN (SELECT Category FROM res2Partial) """)

# Register the temporary table unsoldCategories
unsoldCategoriesDF.createOrReplaceTempView("unsoldCategories")

In [ ]:
# update the result of the second part with a final Union
res2FinalDF = res2PartialDF.union(unsoldCategoriesDF)

In [ ]:
# Store the result of Part 2
res2FinalDF.write\
    .format("csv")\
    .option("header", False)\
    .save(outputPath2)